In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import least_squares
import open3d as o3d
import mediapipe as mp
import json
import trimesh  # For OBJ export

# --- Load images ---
img_front = cv2.imread("samples/nb_front.png")
img_left = cv2.imread("samples/nb_left.png")
img_right = cv2.imread("samples/nb_right.png")
img_front_gray = cv2.cvtColor(img_front, cv2.COLOR_BGR2GRAY)
img_left_gray = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
img_right_gray = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)


def get_sift_matches(img1, img2):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    pts1, pts2 = [], []

    for m, n in matches:
        if m.distance < 0.6 * n.distance:
            good_matches.append(m)
            pts1.append(kp1[m.queryIdx].pt)
            pts2.append(kp2[m.trainIdx].pt)

    return np.array(pts1, dtype=np.float32), np.array(pts2, dtype=np.float32)


def load_camera_parameters(json_file):
    with open(json_file, "r") as f:
        cameras = json.load(f)

    intrinsics_dict = {}
    extrinsics_dict = {}
    projection_matrices = {}

    for cam in cameras:
        name = cam["name"]
        K = np.array(cam["intrinsics"])
        R = np.array(cam["rotation"])
        t = np.array(cam["translation"]).reshape((3, 1))
        Rt = np.hstack((R, t))
        P = K @ Rt

        intrinsics_dict[name] = K
        extrinsics_dict[name] = Rt
        projection_matrices[name] = P

    return intrinsics_dict, extrinsics_dict, projection_matrices


def get_face_landmarks(img):
    mp_face_mesh = mp.solutions.face_mesh
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, refine_landmarks=True
    ) as face_mesh:
        results = face_mesh.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not results.multi_face_landmarks:
            raise ValueError("No face landmarks detected.")
        h, w, _ = img.shape
        landmarks = results.multi_face_landmarks[0].landmark
        return np.array([[lm.x * w, lm.y * h] for lm in landmarks], dtype=np.float32)


def triangulate_landmarks(P1, P2, pts1, pts2):
    pts1 = pts1.T  # shape (2, N)
    pts2 = pts2.T
    pts4D = cv2.triangulatePoints(P1, P2, pts1, pts2)
    pts3D = pts4D[:3] / pts4D[3]
    return pts3D.T


def save_point_cloud(pts3D, filename_prefix="output"):
    # Save as PLY
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts3D)

    # Color coding (red for SIFT, blue for landmarks)
    colors = np.zeros_like(pts3D)
    if len(pts3D) == 1251:  # If combined
        colors[:315] = [1, 0, 0]  # SIFT = red
        colors[315:] = [0, 0, 1]  # Landmarks = blue
    pcd.colors = o3d.utility.Vector3dVector(colors)

    o3d.io.write_point_cloud(f"{filename_prefix}.ply", pcd)

    # Save as OBJ using trimesh
    mesh = trimesh.Trimesh(vertices=pts3D)
    mesh.export(f"{filename_prefix}.obj")

    # Visualize
    o3d.visualization.draw_geometries([pcd])


def reconstruct_face_mesh(pcd, filename_prefix="face_mesh"):
    # 1. Normalize point cloud scale
    pts = np.asarray(pcd.points)
    pts = (pts - np.min(pts)) / (np.max(pts) - np.min(pts))
    pcd.points = o3d.utility.Vector3dVector(pts)

    # 2. Compute normals (critical!)
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
    )
    pcd.orient_normals_consistent_tangent_plane(k=30)

    # 3. Poisson reconstruction with trimming
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9, width=0, scale=1.1, linear_fit=False
    )
    mesh = mesh.select_by_index(np.where(densities > np.quantile(densities, 0.25))[0])

    # 4. Post-processing
    mesh.compute_vertex_normals()
    mesh = mesh.filter_smooth_taubin(number_of_iterations=10)
    mesh.remove_degenerate_triangles()

    # 5. Save
    o3d.io.write_triangle_mesh(f"{filename_prefix}.obj", mesh)
    o3d.io.write_triangle_mesh(f"{filename_prefix}.ply", mesh)
    return mesh


def colorize_point_cloud(pts3D, img, K, Rt):
    pts_hom = np.hstack((pts3D, np.ones((pts3D.shape[0], 1))))
    proj_pts = (K @ Rt @ pts_hom.T).T
    proj_pts = proj_pts[:, :2] / proj_pts[:, 2, np.newaxis]

    h, w, _ = img.shape
    colors = []

    for pt in proj_pts:
        x, y = int(round(pt[0])), int(round(pt[1]))
        if 0 <= x < w and 0 <= y < h:
            color = img[y, x] / 255.0  # BGR to RGB, normalized
            color = color[::-1]  # Convert BGR to RGB
        else:
            color = [0, 0, 0]
        colors.append(color)

    return np.array(colors)


intrinsics, extrinsics, projection_matrices = load_camera_parameters(
    "samples/camera_parameters.json"
)

# Get landmarks
lm_front = get_face_landmarks(img_front)
lm_left = get_face_landmarks(img_left)
lm_right = get_face_landmarks(img_right)

# Triangulate landmarks
pts3D_L = triangulate_landmarks(
    projection_matrices["Camera_Left"],
    projection_matrices["Camera_Front"],
    lm_left,
    lm_front,
)
pts3D_R = triangulate_landmarks(
    projection_matrices["Camera_Right"],
    projection_matrices["Camera_Front"],
    lm_right,
    lm_front,
)

# Get SIFT matches
sift_pts_L, sift_pts_F1 = get_sift_matches(img_left_gray, img_front_gray)
sift_pts_R, sift_pts_F2 = get_sift_matches(img_right_gray, img_front_gray)

# Triangulate SIFT matches
F_L, mask_l = cv2.findFundamentalMat(
    sift_pts_L, sift_pts_F1, cv2.FM_RANSAC, ransacReprojThreshold=2.0, confidence=0.95
)
sift_pts_L = sift_pts_L[mask_l.ravel() == 1]
sift_pts_F1 = sift_pts_F1[mask_l.ravel() == 1]
F_r, mask_r = cv2.findFundamentalMat(
    sift_pts_R, sift_pts_F2, cv2.FM_RANSAC, ransacReprojThreshold=2.0, confidence=0.95
)
sift_pts_R = sift_pts_R[mask_r.ravel() == 1]
sift_pts_F2 = sift_pts_F2[mask_r.ravel() == 1]

pts3D_sift_LF = triangulate_landmarks(
    projection_matrices["Camera_Left"],
    projection_matrices["Camera_Front"],
    sift_pts_L,
    sift_pts_F1,
)
pts3D_sift_RF = triangulate_landmarks(
    projection_matrices["Camera_Right"],
    projection_matrices["Camera_Front"],
    sift_pts_R,
    sift_pts_F2,
)

# Combine all points

pts3D_sift = np.vstack((pts3D_sift_LF, pts3D_sift_RF))
pts3D_land = np.vstack((pts3D_L, pts3D_R))
combined = np.vstack((pts3D_sift[:, :3], pts3D_land[:, :3])).astype(np.float64)

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(pts3D_land)
# Convert to Open3D point cloud with colors from the front image
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pts3D_land)
colors = colorize_point_cloud(
    pts3D_land, img_front, intrinsics["Camera_Front"], extrinsics["Camera_Front"]
)
pcd.colors = o3d.utility.Vector3dVector(colors)


# mesh = reconstruct_face_mesh(pcd)
# o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

mesh = reconstruct_face_mesh(pcd)
o3d.visualization.draw_geometries([mesh])

# face_mesh = reconstruct_face_mesh(combined, "reconstructed_meshed_face")

# Visualize
# o3d.visualization.draw_geometries([face_mesh])

print(
    f"SIFT points: {pts3D_sift.shape}, Landmarks: {pts3D_land.shape}, Combined: {combined.shape}"
)

# Save and visualize all versions
save_point_cloud(pts3D_sift, "sift_points")
save_point_cloud(pts3D_land, "landmarks")
save_point_cloud(combined, "combined_points")

# Additional quality checks
print("First 3 SIFT points:", pts3D_sift[:3])
print("First 3 Landmarks:", pts3D_land[:3])
print("NaN values:", np.isnan(combined).sum())
print("Inf values:", np.isinf(combined).sum())

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import least_squares
import open3d as o3d

# --- Load images ---
img_front = cv2.imread("samples/Camera_Front.png")
img_left = cv2.imread("samples/Camera_Left.png")
img_right = cv2.imread("samples/Camera_Right.png")
img_front_gray = cv2.cvtColor(img_front, cv2.COLOR_BGR2GRAY)
img_left_gray = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
img_right_gray = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

# --- Camera intrinsic matrix (assumed known) ---
# Camera intrinsics
focal_length_mm = 50
sensor_width_mm = 36
image_width_px = 1920
image_height_px = 1080

# Calculate focal length in pixels
focal_length_px = (focal_length_mm / sensor_width_mm) * image_width_px

# Camera intrinsics matrix
K = np.array(
    [
        [2666.666666666667, 0, 960],
        [0, 2250, 540],
        [0, 0, 1],
    ]
)


# --- Feature detection & matching function ---
def match_features(img1_gray, img2_gray):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(img1_gray, None)
    kp2, desc2 = sift.detectAndCompute(img2_gray, None)

    # FLANN parameters for SIFT
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=9)
    search_params = dict(checks=50)  # Higher = more accurate but slower

    # Initialize FLANN matcher
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Perform KNN matching
    matches = flann.knnMatch(desc1, desc2, k=2)

    # Apply Lowe's ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.88 * n.distance:
            good.append(m)

    # Extract matched keypoints
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good])

    return pts1, pts2


# --- Match Front-Left and Front-Right ---
pts_front_l, pts_left = match_features(img_front_gray, img_left_gray)
pts_front_r, pts_right = match_features(img_front_gray, img_right_gray)

print(f"Matched Front-Left points: {len(pts_front_l)}")
print(f"Matched Front-Right points: {len(pts_front_r)}")


# --- Epipolar lines drawing ---
def draw_epilines(img1, img2, pts1, pts2, F):
    h, w = img1.shape[:2]
    lines = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
    lines = lines.reshape(-1, 3)
    img1_epi = img1.copy()
    for r, pt in zip(lines, pts1):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        if r[1] != 0:
            x0, y0 = map(int, [0, -r[2] / r[1]])
            x1, y1 = map(int, [w, -(r[2] + r[0] * w) / r[1]])
            cv2.line(img1_epi, (x0, y0), (x1, y1), color, 1)
        cv2.circle(img1_epi, tuple(map(int, pt)), 4, color, -1)
    return img1_epi


# --- Fundamental Matrices ---
F_l, mask_l = cv2.findFundamentalMat(pts_front_l, pts_left, cv2.FM_RANSAC)
F_r, mask_r = cv2.findFundamentalMat(pts_front_r, pts_right, cv2.FM_RANSAC)

# Filter inliers
pts_front_l, pts_left = pts_front_l[mask_l.ravel() == 1], pts_left[mask_l.ravel() == 1]
pts_front_r, pts_right = (
    pts_front_r[mask_r.ravel() == 1],
    pts_right[mask_r.ravel() == 1],
)

print(f"Inlier Front-Left matches: {len(pts_front_l)}")
print(f"Inlier Front-Right matches: {len(pts_front_r)}")

# --- Draw epipolar lines ---
front_left_epi = draw_epilines(img_front, img_left, pts_front_l, pts_left, F_l)
left_epi = draw_epilines(img_left, img_front, pts_left, pts_front_l, F_l.T)
front_right_epi = draw_epilines(img_front, img_right, pts_front_r, pts_right, F_r)
right_epi = draw_epilines(img_right, img_front, pts_right, pts_front_r, F_r.T)

plt.figure(figsize=(12, 10))
plt.subplot(221), plt.imshow(
    cv2.cvtColor(front_left_epi, cv2.COLOR_BGR2RGB)
), plt.title("Front Epilines - Left"), plt.axis("off")
plt.subplot(222), plt.imshow(cv2.cvtColor(left_epi, cv2.COLOR_BGR2RGB)), plt.title(
    "Left Epilines"
), plt.axis("off")
plt.subplot(223), plt.imshow(
    cv2.cvtColor(front_right_epi, cv2.COLOR_BGR2RGB)
), plt.title("Front Epilines - Right"), plt.axis("off")
plt.subplot(224), plt.imshow(cv2.cvtColor(right_epi, cv2.COLOR_BGR2RGB)), plt.title(
    "Right Epilines"
), plt.axis("off")
plt.tight_layout()
plt.savefig("output_plot.png")  # Saves the plot as an image file
plt.close()  # Clears the figure to free memory
# plt.show()

# --- Triangulation ---
E_l = K.T @ F_l @ K
_, R_l, t_l, _ = cv2.recoverPose(E_l, pts_front_l, pts_left, K)

E_r = K.T @ F_r @ K
_, R_r, t_r, _ = cv2.recoverPose(E_r, pts_front_r, pts_right, K)
print(f"-Right rotation: ", R_l)
P0 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
P_l = K @ np.hstack((R_l, t_l))
P_r = K @ np.hstack((R_r, t_r))

pts4D_l = cv2.triangulatePoints(P0, P_l, pts_front_l.T, pts_left.T)
pts4D_r = cv2.triangulatePoints(P0, P_r, pts_front_r.T, pts_right.T)

pts3D_l = (pts4D_l[:3] / pts4D_l[3]).T
pts3D_r = (pts4D_r[:3] / pts4D_r[3]).T
pts3D = np.vstack((pts3D_l, pts3D_r))


# --- Optional: Bundle Adjustment ---
def reprojection_error(params, pts_3d, pts_2d, K):
    rvec = params[:3].astype(np.float64)
    tvec = params[3:6].astype(np.float64)
    proj_pts, _ = cv2.projectPoints(pts_3d.astype(np.float64), rvec, tvec, K, None)
    return (proj_pts.squeeze() - pts_2d).ravel()


init_params = np.zeros(6)
ba_result = least_squares(reprojection_error, init_params, args=(pts3D_l, pts_left, K))


# --- Reprojection Visualization ---
def plot_reprojection(pts_actual, pts_reproj, title):
    plt.figure(figsize=(6, 6))
    plt.scatter(pts_actual[:, 0], pts_actual[:, 1], c="blue", label="Actual 2D Points")
    plt.scatter(
        pts_reproj[:, 0],
        pts_reproj[:, 1],
        c="red",
        marker="+",
        label="Reprojected Points",
    )
    for i in range(len(pts_actual)):
        plt.plot(
            [pts_actual[i, 0], pts_reproj[i, 0]],
            [pts_actual[i, 1], pts_reproj[i, 1]],
            "gray",
            linewidth=0.5,
        )
    plt.gca().invert_yaxis()
    plt.legend()
    plt.title(title)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.savefig("output_projection.png")
    # Saves the plot as an image file
    plt.close()  # Clears the figure to free memory
    # plt.show()


rvec_opt = ba_result.x[:3]
tvec_opt = ba_result.x[3:6]
pts2d_reproj, _ = cv2.projectPoints(
    pts3D_l.astype(np.float64), rvec_opt, tvec_opt, K, None
)
pts2d_reproj = pts2d_reproj.squeeze()
plot_reprojection(pts_left, pts2d_reproj, "Reprojection vs Actual (Left View)")


# --- Save as PLY file ---
def save_to_ply(filename, points):
    with open(filename, "w") as f:
        f.write("ply\nformat ascii 1.0\nelement vertex {}\n".format(len(points)))
        f.write("property float x\nproperty float y\nproperty float z\nend_header\n")
        for p in points:
            f.write("{} {} {}\n".format(p[0], p[1], p[2]))


save_to_ply("reconstructed.ply", pts3D)
print("PLY file saved as 'reconstructed.ply' with {} points.".format(len(pts3D)))

# --- Visualize with Open3D (sparse) ---
pcl = o3d.geometry.PointCloud()
pcl.points = o3d.utility.Vector3dVector(pts3D)


# --- Add camera frustums ---
def create_camera_frustum(scale=0.3, color=[1, 0, 0], line_width=2.0):
    """Create a compact camera frustum visualization

    Args:
        scale: Size of the frustum (default 0.3 is good for face reconstruction)
        color: RGB color of the frustum lines
        line_width: Visual thickness of the lines

    Returns:
        Open3D LineSet representing the camera frustum
    """
    frustum = o3d.geometry.LineSet()

    # More compact pyramid geometry (base is now closer to apex)
    points = (
        np.array(
            [
                [0, 0, 0],  # Apex (camera center)
                [0.5, 0.5, 1],  # Top-right
                [-0.5, 0.5, 1],  # Top-left
                [-0.5, -0.5, 1],  # Bottom-left
                [0.5, -0.5, 1],  # Bottom-right
            ]
        )
        * scale
    )

    # Pyramid edges
    lines = [
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],  # Apex to corners
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 1],  # Base rectangle
    ]

    frustum.points = o3d.utility.Vector3dVector(points)
    frustum.lines = o3d.utility.Vector2iVector(lines)

    # Make lines more visible
    frustum.paint_uniform_color(color)

    # For Open3D versions that support line width
    if hasattr(frustum, "line_width"):
        frustum.line_width = line_width

    return frustum


# Create smaller frustums (scale=0.3 instead of 5)
camera_front = create_camera_frustum(scale=0.3, color=[1, 0, 0])  # Red
camera_left = create_camera_frustum(scale=0.3, color=[0, 1, 0])  # Green
camera_right = create_camera_frustum(scale=0.3, color=[0, 0, 1])  # Blue

# Apply camera poses
T_left = np.eye(4)
T_left[:3, :3] = R_l
T_left[:3, 3] = t_l.ravel()
camera_left = camera_left.transform(T_left)

T_right = np.eye(4)
T_right[:3, :3] = R_r
T_right[:3, 3] = t_r.ravel()
camera_right = camera_right.transform(T_right)

# Visualize with better viewing parameters
vis = o3d.visualization.Visualizer()
vis.create_window(
    width=800, height=600, window_name="3D Face Reconstruction with Camera Frustums"
)

# Add geometries
vis.add_geometry(pcl)
vis.add_geometry(camera_front)
vis.add_geometry(camera_left)
vis.add_geometry(camera_right)

# Set better viewing angle
ctrl = vis.get_view_control()
ctrl.set_front([0, 0, -1])  # Looking along negative z-axis
ctrl.set_up([0, -1, 0])  # Y-axis is up
ctrl.set_zoom(0.8)  # Slightly zoomed out

# Custom rendering options
opt = vis.get_render_option()
opt.background_color = np.array([0.9, 0.9, 0.9])  # Light gray background
opt.light_on = True

vis.run()
vis.destroy_window()

# --- Dense Stereo Matching ---

# Define stereo matcher
stereo = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=192,
    blockSize=5,
    P1=8 * 3 * 5**2,
    P2=32 * 3 * 5**2,
    disp12MaxDiff=5,
    uniquenessRatio=15,
    speckleWindowSize=50,
    speckleRange=32,
)

# Compute disparity for front-right
disp_fr = stereo.compute(img_front_gray, img_right_gray).astype(np.float32) / 16.0
# Compute disparity for front-left
disp_fl = stereo.compute(img_front_gray, img_left_gray).astype(np.float32) / 16.0

# Display disparity maps
plt.figure()
plt.imshow(disp_fr, cmap="jet")
plt.colorbar()
plt.title("Disparity Map Front-Right")
plt.savefig("disparity_map_fr.png")
plt.close()

plt.figure()
plt.imshow(disp_fl, cmap="jet")
plt.colorbar()
plt.title("Disparity Map Front-Left")
plt.savefig("disparity_map_fl.png")
plt.close()

# Camera intrinsics (dummy values - replace with calibration data)


# Reprojection matrix
h, w = img_front_gray.shape
Q = np.float32(
    [[1, 0, 0, -K[0, 2]], [0, -1, 0, K[1, 2]], [0, 0, 0, K[0, 0]], [0, 0, 1, 0]]
)

# Reproject to 3D for both
points_3d_fr = cv2.reprojectImageTo3D(disp_fr, Q)
points_3d_fl = cv2.reprojectImageTo3D(disp_fl, Q)
colors_fr = cv2.cvtColor(img_front, cv2.COLOR_BGR2RGB)
colors_fl = cv2.cvtColor(img_front, cv2.COLOR_BGR2RGB)

mask_fr = (disp_fr > disp_fr.min()) & (points_3d_fr[:, :, 2] < 1000)
mask_fl = (disp_fl > disp_fl.min()) & (points_3d_fl[:, :, 2] < 1000)

out_points_fr = points_3d_fr[mask_fr]
out_colors_fr = colors_fr[mask_fr]
out_points_fl = points_3d_fl[mask_fl]
out_colors_fl = colors_fl[mask_fl]

# Merge point clouds
all_points = np.vstack((out_points_fr, out_points_fl))
all_colors = np.vstack((out_colors_fr, out_colors_fl))

print(f"Generated {len(all_points)} 3D points")

# Create point cloud
pcl_dense = o3d.geometry.PointCloud()
pcl_dense.points = o3d.utility.Vector3dVector(all_points)
pcl_dense.colors = o3d.utility.Vector3dVector(all_colors / 255.0)

# Visualize dense point cloud
o3d.visualization.draw_geometries(
    [pcl_dense],
    window_name="Dense 3D Reconstruction",
    width=800,
    height=600,
    point_show_normal=False,
)

# Mesh generation
pcl_dense.estimate_normals()
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcl_dense, depth=7
)
mesh.remove_vertices_by_mask(densities < np.quantile(densities, 0.1))
o3d.visualization.draw_geometries([mesh], window_name="Combined Mesh")

# Save outputs
o3d.io.write_point_cloud("dense_reconstruction.ply", pcl_dense)
o3d.io.write_triangle_mesh("dense_mesh.obj", mesh)
print("Saved both point cloud and mesh")

# Create 2D frontal projection
vis = o3d.visualization.Visualizer()
vis.create_window(width=w, height=h)
vis.add_geometry(pcl_dense)
ctr = vis.get_view_control()
bbox = pcl_dense.get_axis_aligned_bounding_box()
ctr.fit_bounds(bbox.get_min_bound(), bbox.get_max_bound())

vis.poll_events()
vis.update_renderer()
vis.capture_screen_image("frontal_view.png", do_render=True)
vis.destroy_window()
print("Saved frontal view as 'frontal_view.png'")

Matched Front-Left points: 478
Matched Front-Right points: 363
Inlier Front-Left matches: 279
Inlier Front-Right matches: 171
-Right rotation:  [[ 8.79549157e-01  6.73968132e-03 -4.75760293e-01]
 [-5.48580285e-03  9.99976856e-01  4.02406861e-03]
 [ 4.75776403e-01 -9.29438979e-04  8.79565774e-01]]
PLY file saved as 'reconstructed.ply' with 450 points.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Generated 217559 3D points
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Saved both point cloud and mesh
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


AttributeError: 'open3d.cpu.pybind.visualization.ViewControl' object has no attribute 'fit_bounds'